#### Step 5: Train/Test Split**

In [5]:
# Read processed excel
import pandas as pd
df = pd.read_csv(r"D:\DATA SCIENCE\SCHOOL PROJECTS\MACHINE LEARNING\data\processed\crypto_labeled_features.csv")



## MAY ADD FEATURE COLS???**

In [6]:
# Define features and target

target_col = "label"  # column with your Buy/Sell/Hold labels
feature_cols = df.columns.difference([target_col, "future_return", "open_time", "close_time"])

X = df[feature_cols]
y = df[target_col]


# Train/Validation/Test split

train_size = int(len(df) * 0.70)
val_size = int(len(df) * 0.15)
test_start = train_size + val_size

X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]

X_val = X.iloc[train_size:test_start]
y_val = y.iloc[train_size:test_start]

X_test = X.iloc[test_start:]
y_test = y.iloc[test_start:]


# Print summary

print(f"Train: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation: {len(X_val)} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"Test: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")


Train: 560 samples (69.9%)
Validation: 120 samples (15.0%)
Test: 121 samples (15.1%)


#### Step 6: Model Training

In [7]:
# Random Forest\
# LightGBM\
#  XGBoost\
#   CatBoost\
#   LSTM/GRU (bonus)

#Example:

#``` python
#from xgboost import XGBClassifier

# model = XGBClassifier()
# model.fit(X_train, y_train)

##### Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', class_weight='balanced')
model_lr.fit(X_train, y_train)


C:\Users\Tessy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Tessy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'multinomial'


In [21]:
y_val_pred = model_lr.predict(X_val_scaled)

print("=== Classification Report ===")
print(classification_report(y_val, y_val_pred, digits=4))
print("=== Confusion Matrix ===")
print(confusion_matrix(y_val, y_val_pred))

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.0672    0.8889    0.1250         9
           1     0.0000    0.0000    0.0000        94
           2     0.0000    0.0000    0.0000        17

    accuracy                         0.0667       120
   macro avg     0.0224    0.2963    0.0417       120
weighted avg     0.0050    0.0667    0.0094       120

=== Confusion Matrix ===
[[ 8  0  1]
 [94  0  0]
 [17  0  0]]


C:\Users\Tessy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Tessy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Tessy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_clas

In [10]:
# np.exp(result.params)

In [11]:
import statsmodels.api as sm

# Create intercept term required for sm.Logit, see documentation for more information
# X = sm.add_constant(X)

# Fit model
# logit_model = sm.Logit(y, X)

# Get results of the fit
# result = logit_model.fit()

In [16]:
# ==========================================
# CATBOOST MODEL FOR PRICE DIRECTION PREDICTION
# ==========================================

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# ------------------------------------------
# 1. Load or assume df_labeled already exists
# ------------------------------------------
# df_labeled = your engineered + labeled DataFrame

# ------------------------------------------
# 2. Define features
# ------------------------------------------
features = [
    "close_pct_1d", "close_pct_7d", "vol_7d", "vol_30d",
    "rsi", "sma_20", "sma_50", "sma_200",
    "bb_high", "bb_mid", "bb_low",
    "stoch_k", "stoch_d",
    "trades_7d_avg", "trades_30d_avg",
    "volume_7d_avg", "volume_30d_avg"
]

X = df[features]
y = df["label"]

# ------------------------------------------
# 3. Train-test split (do NOT shuffle time-series)
# ------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# ------------------------------------------
# 4. Build & train CatBoost model
# ------------------------------------------
model = CatBoostClassifier(
    iterations=800,
    depth=8,
    learning_rate=0.03,
    loss_function="Logloss",
    verbose=False
)

print("Training CatBoost...")
model.fit(X_train, y_train)

# ------------------------------------------
# 5. Evaluate model
# ------------------------------------------
y_pred = model.predict(X_test)

print("\n📌 CATBOOST RESULTS")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Training CatBoost...


CatBoostError: catboost/private/libs/target/target_converter.cpp:410: Target with classes must contain only 2 unique values for binary classification

In [18]:
!pip install tensorflow-cpu


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Tessy\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Tessy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/331.9 MB 270.5 kB/s eta 0:20:25
   ---------------------------------------- 0.5/331.9 MB 270.5 kB/s eta 0:20:25
   ---------------------------------------- 0.5/331.9 MB 270.5 kB/s eta 0:20:25
   ---------------------------------------- 0.

In [20]:
import tensorflow as tf



ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
# ==========================================
# LSTM MODEL FOR PRICE DIRECTION PREDICTION
# ==========================================

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report

# ------------------------------------------
# 1. Assume df_labeled already exists
# ------------------------------------------

features = [
    "close_pct_1d", "close_pct_7d", "vol_7d", "vol_30d",
    "rsi", "sma_20", "sma_50", "sma_200",
    "bb_high", "bb_mid", "bb_low",
    "stoch_k", "stoch_d",
    "trades_7d_avg", "trades_30d_avg",
    "volume_7d_avg", "volume_30d_avg"
]

label_col = "label"

# ------------------------------------------
# 2. Sequence creation
# ------------------------------------------
def create_sequences(df, feature_cols, label_col, seq_len=30):
    X_list, y_list = [], []

    data = df[feature_cols].values
    labels = df[label_col].values

    for i in range(seq_len, len(df)):
        X_list.append(data[i-seq_len:i])
        y_list.append(labels[i])

    return np.array(X_list), np.array(y_list)

seq_len = 30
X_seq, y_seq = create_sequences(df_labeled, features, label_col, seq_len)

# ------------------------------------------
# 3. Train-test split
# ------------------------------------------
split = int(len(X_seq) * 0.8)

X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]


#Build LSTM model

model_lstm = models.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=(seq_len, X_train.shape[2])),
    layers.LSTM(32),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_lstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_lstm.summary()


#Train model

history = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=12,
    batch_size=32,
    verbose=1
)


#Evaluate model

pred_probs = model_lstm.predict(X_test)
y_pred = (pred_probs > 0.5).astype(int)

print("\n📌 LSTM RESULTS")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


ModuleNotFoundError: No module named 'tensorflow'

In [12]:
from catboost import CatBoostClassifier


# ----------------------
# 3. Train CatBoost
# ----------------------
model = CatBoostClassifier(
    iterations=800,
    depth=8,
    learning_rate=0.03,
    loss_function="Logloss",
    verbose=100
)

model.fit(X_train, y_train)



CatBoostError: catboost/private/libs/target/target_converter.cpp:410: Target with classes must contain only 2 unique values for binary classification

In [14]:
history = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=12,
    batch_size=32
)


NameError: name 'model_lstm' is not defined

In [13]:
history = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=12,
    batch_size=32
)


NameError: name 'model_lstm' is not defined